In [2]:
import os
from glob import glob
import pandas as pd

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all
from urllib.parse import unquote

You are using wmfdata 0.1.0 (latest).

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [9]:
filename = '../../data/raw/articles/2019/Tiger 2.0 - article page stats.xlsx'

cols2skip = [0,2,3]
cols = [i for i in range(10) if i not in cols2skip]
dfs = pd.read_excel(filename, sheet_name=None, usecols=cols, header=None) #skiprows=1

df = pd.concat([df.assign(name=n) for n,df in dfs.items()], sort=True)
df.columns = ['page_title', 'language_name']

In [36]:
#check the langauage names (aka sheets by wiki) and make sure we can identify each language
#pnbwiki	Western Punjabi	https://pnb.wikipedia.org/
#pawiki	Punjabi	Punjabi(Gurumukhi)	https://pa.wikipedia.org/

df['language_name'] = df['language_name'].replace('Bengali', 'Bangla')
df['language_name'] = df['language_name'].replace('Punjabi(Gurumukhi)', 'Punjabi')

lang_names =tuple(df['language_name'].unique())
len(lang_names)

15

In [41]:
ci = wmf.hive.run("""
SELECT  database_code, language_name, language_code
FROM canonical_data.wikis
WHERE language_name IN {lang_names} AND database_group = 'wikipedia'
""".format(lang_names=lang_names))

In [42]:
#merge
df_ci = df.merge(ci, how="left", on=['language_name'])

In [44]:
df_ci.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9772 entries, 0 to 9771
Data columns (total 4 columns):
page_title       9173 non-null object
language_name    9772 non-null object
database_code    9772 non-null object
language_code    9772 non-null object
dtypes: object(4)
memory usage: 381.7+ KB


In [45]:
df_ci.to_csv("../../data/raw/articles/2019/clean/articles.csv", sep=',', encoding = 'utf-8', index=False)